# Titanic 생존
`titanic.ipynb`

## 참고할 제언
- imputer 사용  > age 결측치 채우고 , embarked 수동으로 결측치 세우기를 함. 
- OHE 원핫인코딩 > 범주형 데이터를 여러개의 컬럼으로 변환 
- Pros -> 숫자 0, 1, 2 는 크기 순서가 생겨서 해석이 잘못될 수 있음
- Cons -> 카테고리가 많으면 차원이 늘어남 
- MICE: 결측치가 있는 변수를 다른 변수를 이용해 반복적으로 예측한다. 


In [85]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

titanic = sns.load_dataset('titanic')

1. 데이터 준비 및 탐색 (EDA)
-  데이터 불러오기: train.csv, test.csv 읽기
-  기초 확인: head(), info(), describe()로 데이터 구조 파악
-  결측치 확인: Age, Cabin, Embarked 등 결측치 처리 전략 수립
-  타겟 분포 확인: Survived 비율(0/1)

In [86]:
# 타이타닉(데이터 프레임 ) 데이터셋 불러오기
# (pandas에 기본 내장된 예제 데이터셋)
import seaborn as sns
import pandas as pd

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

# 처음 5개 행 보기
print("타이타닉 데이터 미리보기:")
print(titanic.head())

# 데이터셋 정보 확인
print("\n데이터셋 정보:")
print(titanic.info())

# 데이터 요약 통계
print("\n데이터 요약 통계:")
print(titanic.describe())

타이타닉 데이터 미리보기:
   survived  pclass     sex   age  ...  deck  embark_town  alive  alone
0         0       3    male  22.0  ...   NaN  Southampton     no  False
1         1       1  female  38.0  ...     C    Cherbourg    yes  False
2         1       3  female  26.0  ...   NaN  Southampton    yes   True
3         1       1  female  35.0  ...     C  Southampton    yes  False
4         0       3    male  35.0  ...   NaN  Southampton     no   True

[5 rows x 15 columns]

데이터셋 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-nu

In [87]:
# 결측치 확인 
# Col 결측치 합계  
df = titanic
titanic.isnull().sum()
# 승객 등급 등장 빈도 (1, 2, 3 등석)
df['class'].value_counts()[0:3]
# 성별 빈도(남/녀)
df['sex'].value_counts()
# 생존 여부 빈도(생존자/사망자)
df['alive'].value_counts()


alive
no     549
yes    342
Name: count, dtype: int64

In [88]:
# 결측치 처리 
titanic = sns.load_dataset('titanic')
#결측치 확인 
missing = titanic.isna().sum()
#결측이 있는 항목만 확인 
missing[missing > 0]
#결측비율 
titanic.isna().mean() * 100

##--> 결측 확인 결과 embarked, deck, age, embarked_town에 결측치가 존재함을 확인함.


survived        0.000000
pclass          0.000000
sex             0.000000
age            19.865320
sibsp           0.000000
parch           0.000000
fare            0.000000
embarked        0.224467
class           0.000000
who             0.000000
adult_male      0.000000
deck           77.216611
embark_town     0.224467
alive           0.000000
alone           0.000000
dtype: float64

In [89]:
# 타겟 분포 확인 
#성별에 따른 생존율 survived 
survival_rate = df.groupby('sex')['survived'].mean()
#더 복잡한 톡예 가능 
titanic.groupby('sex').agg({'survived':['mean']})
#승객 등급에 따른 생존율 
survival_rate1 = df.groupby('pclass')['survived'].mean()
print(survival_rate, survival_rate1)
#전체 인원중 생존자 수 

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64 pclass
1    0.629630
2    0.472826
3    0.242363
Name: survived, dtype: float64


In [90]:
# 결측치 처리
df['age'].fillna(df['age'].median(), inplace=True) # 중간값으로 결측치 대체 
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)#최빈값으로 결측치 대체,시리즈의 첫번째 
#값을 선택해서 mode()의 결과가 여러개일 경우, 첫번째 값만 사용 
# 범주형변수 인코딩 -> 숫자형 변환
df['sex'] = df['sex'].map({'male':0, 'female':1})
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)


# KNN 분류 
- KNN은 거리 기반이므로 스케일링이 필요함 

In [91]:
# 데이터 분리: 생존율을 예측하려고 함. 
X= df.drop('survived',axis=1) #예측에 사용되는 독립변수 
y= df['survived'] #이 열만 종속변수(예측하려는 값) 

# X와 y를 각각 훈련세트와 테스트 세트로 나눈다.
# 전체데이터의 20을 테스트로 사용 나머지는 훈련데이터 
#seed 설정: 코드 다시실행해도 동일한 결과 얻도록 무작위 추출 
#startify=y: y에 해당하는 클래스를 훈련세트와 테스트세트에서 동일하게 적용된다. 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state= 42, stratify=y 
)

In [95]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --------------------------------------------------------
# 1. 데이터 준비
# --------------------------------------------------------
X = df.drop('survived', axis=1)
y = df['survived']

# 숫자형 / 범주형 컬럼 나누기
num_cols = X.select_dtypes(include=['int64', 'float64', 'bool']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

# --------------------------------------------------------
# 2. 전처리 정의
# --------------------------------------------------------
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),               # 숫자형 → 스케일링
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)  # 범주형 → 원핫인코딩
])

# --------------------------------------------------------
# 3. 파이프라인 구성 (전처리 + 모델)
# --------------------------------------------------------
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier())
])

# --------------------------------------------------------
# 4. 하이퍼파라미터 탐색
# --------------------------------------------------------
param_grid = {
    'knn__n_neighbors': range(3, 21),      
    'knn__weights': ['uniform', 'distance']
}

grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid.fit(X, y)

# --------------------------------------------------------
# 5. 최적 모델 확인
# --------------------------------------------------------
print("Best Parameters:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

# 최적 모델
best_knn = grid.best_estimator_

Best Parameters: {'knn__n_neighbors': 4, 'knn__weights': 'distance'}
Best CV Score: 0.9663235201807796


### KNN 결과 
- k 값이 4이고, 이웃노드의 가중치를 거리에 반비예하게 설정한 모델이 가장 놓은 성능을 보임 
- 96.63의 정확도를 보임 

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    int64   
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   class        891 non-null    category
 8   who          891 non-null    object  
 9   adult_male   891 non-null    bool    
 10  deck         203 non-null    category
 11  embark_town  889 non-null    object  
 12  alive        891 non-null    object  
 13  alone        891 non-null    bool    
 14  embarked_Q   891 non-null    bool    
 15  embarked_S   891 non-null    bool    
dtypes: bool(4), category(2), float64(2), int64(5), object(3)
memory usage: 75.4+ KB


# Logistic 회귀 분류 

In [ ]:
from sklearn.linear_model import LogisticRegression


# 결정트리 분류 